In [1]:
from google.cloud import bigquery
import os
from dotenv import load_dotenv
import pandas as pd
import logging
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import matplotlib.pyplot as plt


/Users/ecesuolgun/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
#Logging configuration
logging.basicConfig(
    level=logging.INFO, 
    format="%(asctime)s - %(levelname)s - %(message)s", 
)

In [3]:
load_dotenv()
CREDENTIALS_PATH = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
if os.path.exists('clustered_events.feather'):
    logging.info("Local file found, loading...")
    df = pd.read_feather('clustered_events.feather')
else:
    logging.info("Local file not found, fetching from BigQuery...")
    query = """
    SELECT *
    FROM `apps-interview-project.analytics_v2.clustered_events`
    """
    client = bigquery.Client.from_service_account_json(CREDENTIALS_PATH)
    df = client.query(query).to_dataframe()
    df.to_feather('clustered_events.feather')

2025-04-30 20:32:34,506 - INFO - Local file found, loading...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13497147 entries, 0 to 13497146
Data columns (total 12 columns):
 #   Column              Dtype              
---  ------              -----              
 0   event_date          dbdate             
 1   event_timestamp     Int64              
 2   installed_datetime  datetime64[us, UTC]
 3   platform            object             
 4   country             object             
 5   user_id             Int64              
 6   event_name          object             
 7   exp_group           Int64              
 8   is_online           Int64              
 9   ad_revenue          float64            
 10  ad_type             object             
 11  level_name          Int64              
dtypes: Int64(5), datetime64[us, UTC](1), dbdate(1), float64(1), object(4)
memory usage: 1.3+ GB


In [5]:
df['ad_revenue'].describe()

count    1.100898e+07
mean     4.398796e-03
std      2.231524e-02
min      2.000000e-08
25%      1.501600e-04
50%      3.903743e-04
75%      1.439574e-03
max      5.086236e+00
Name: ad_revenue, dtype: float64

In [6]:
df.head()

,event_date,event_timestamp,installed_datetime,platform,country,user_id,event_name,exp_group,is_online,ad_revenue,ad_type,level_name
0,2024-07-11,1720690917684018,2024-07-11 09:28:05.413000+00:00,android,tr,-2664840524922734347,AdImpressionRevenue,-8619511817166493278,<NA>,0.000158,banner,<NA>
1,2024-07-11,1720680837216009,2024-05-16 13:17:21.290000+00:00,ios,gb,5139130612280951285,AdImpressionRevenue,<NA>,<NA>,0.000065,banner,<NA>
2,2024-07-11,1720737075021002,2024-01-28 02:53:03.901000+00:00,android,us,4131906675711972341,AdImpressionRevenue,<NA>,<NA>,0.005400,banner,<NA>
3,2024-07-11,1720660341439008,2024-06-24 15:36:56.422000+00:00,android,us,7861136691032275445,AdImpressionRevenue,-8619511817166493278,<NA>,0.034931,interstitial,<NA>
4,2024-07-11,1720713454918000,2024-07-11 12:45:51.602000+00:00,android,tr,5310866246176149238,AdImpressionRevenue,1046097541929355958,<NA>,0.000648,banner,<NA>
